In [ ]:
#!pip install geemap
#!pip install ipyleaflet


In [1]:
import ee
import ee; ee.Initialize()
import geemap
import os
import ipyleaflet
import numpy as np

In [27]:
js='''
var dataset = ee.ImageCollection('USDA/NASS/CDL')
                  .filter(ee.Filter.date('2018-01-01', '2019-12-31'))
                  .first();
var cropLandcover = dataset.select('cultivated');
Map.setCenter(-100.55, 40.71, 4);
Map.addLayer(cropLandcover, {}, 'Crop Landcover');
'''


In [28]:
#geemap.js_snippet_to_py(js, add_new_cell=True)

In [3]:
#state 
stateBoundary = geemap.ee.FeatureCollection('TIGER/2018/States').select('NAME')\
.filter(geemap.ee.Filter.eq('NAME', 'Minnesota'))
#Map.addLayer(MinnesotaBoundary, {}, 'Minnesota')

#county 
CountyBoundary = geemap.ee.FeatureCollection('TIGER/2018/Counties').select('NAME')\
.filter(geemap.ee.Filter.eq('NAME', 'Jackson'))
#Map.addLayer(CountyBoundary, {}, 'County')

#area of study
cntyGeom = CountyBoundary.geometry();


In [4]:
def clip1(featurecollection):
  return featurecollection.intersection(cntyGeom, 1)

In [5]:
region = stateBoundary.map(clip1)

In [ ]:
Map = geemap.Map(center=(43.65, -95.17), zoom=10)
Map.addLayer(region, {}, 'region')
Map

In [6]:
def addArea(feature):
  return feature.set({'areaHa': feature.geometry().area().divide(100 * 100)})


In [7]:
areaAdded = region.map(addArea)

In [8]:
rgnGeom = areaAdded.geometry();

In [9]:
loc = geemap.ee.Geometry(rgnGeom)
collection = ee.ImageCollection('USDA/NAIP/DOQQ') \
    .filterBounds(loc) \
    .filterDate('2017-01-01', '2018-01-01') \
    .filter(ee.Filter.listContains("system:band_names", "N"))

In [ ]:
Map = geemap.Map(center=(43.65, -95.17), zoom=10)
Map.addLayer(collection, {}, 'collection')
Map

In [ ]:
#Number Of Images 
n = collection.size().getInfo();
print(n)

# bands
# image = collection.first()
# print('Band names: ', image.getInfo())

In [10]:
# Create the NDVI and NDWI spectral indices.
def addNDVI(image):
  ndvi = image.normalizedDifference(['N', 'R']).rename('NDVI')
  return image.addBands(ndvi)

def addNDWI(image):
  ndwi = image.normalizedDifference(['G', 'N']).rename('NDWI')
  return image.addBands(ndwi)


In [13]:

# apply functions to  collection
ndvi = collection.map(addNDVI)
ndwi = collection.map(addNDWI)


In [ ]:
Map = geemap.Map(center=(43.65, -95.17), zoom=10)
Map.addLayer(ndwi.select('NDWI'), {'min': 0.5, 'max': 0.6, 'palette': ['4E565B', '2B94D8']}, 'ndwi')
Map.addLayer(ndvi.select('NDVI'), {'min': 0.2, 'max': 0.5, 'palette': ['472D67', 'F9F503']}, 'ndvi')
Map

In [ ]:
# Create the bare spectral indices.
# Create some binary images from thresholds on the indices.
# This threshold is designed to detect bare land.
def addbare1(image):
  bare1 = image.normalizedDifference(['N', 'R']).lt(0.2).And(image.normalizedDifference(['G', 'N']).lt(0.3)).rename('bare1')
  return image.addBands(bare1)
# This detects bare land with lower sensitivity. It also detects shadows.
def addbare2(image):
  bare2 = image.normalizedDifference(['N', 'R']).lt(0.2).And(image.normalizedDifference(['G', 'N']).lt(0.8)).rename('bare2')
  return image.addBands(bare2)


In [ ]:
# # apply functions to  collection
# bare1 = collection.map(addbare1)
# bare2 = collection.map(addbare2)


In [ ]:
# Map = geemap.Map(center=(43.65, -95.17), zoom=10)
# # Visualize bare areas 
# Map.addLayer(bare1.select('bare1'), {'min': 0.6, 'max': 1, 'palette': ['F4EDE7', '904C13']}, 'bare1')
# Map.addLayer(bare2.select('bare2'), {'min': 0.6, 'max': 1, 'palette': ['F4EDE7', '904C13']}, 'bare2')
# # NDWI > 0.5 is water.  Visualize it with a blue palette.
# Map.addLayer(ndwi.select('NDWI'), {'min': 0.5, 'palette': ['4E565B', '2B94D8']}, 'ndwi')
# # NDVI > 0.2 is vegetation.  Visualize it with a green palette.
# Map.addLayer(ndvi.select('NDVI'), {'min': 0.2, 'max': 0.6, 'palette': ['472D67', 'F9F503']}, 'ndvi')
# Map

In [19]:
median = ndvi.median()

bandNames = median.bandNames()
print('Band names: ', bandNames.getInfo())


Band names:  ['R', 'G', 'B', 'N', 'NDVI']


In [ ]:
Map = geemap.Map(center=(43.65, -95.17), zoom=10)
Map.addLayer(median.select('NDVI'), {'min': 0.2, 'max': 0.5, 'palette': ['472D67', 'F9F503']}, 'median ndvi', False)
Map

In [ ]:
#unsupervised classification

region = Map.user_roi
#region = ee.Geometry.Rectangle([-95.1550,43.7018, -95.1643,43.6961])
region = ee.Geometry.Point([ -95.1601,43.7052]).buffer(10000)

# Make the training dataset.
training = median.select('NDVI').sample(**{
    'region': region,
    'numPixels': 5000,
    'seed': 0,
    'scale': 30,
    'geometries': True  
})

Map.addLayer(training, {}, 'training', False)


In [ ]:
# Instantiate the clusterer and train it.
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

# Cluster the input using the trained clusterer.
result = median.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

In [ ]:
bandNames = result.bandNames()
print('Band names: ', bandNames.getInfo())

In [ ]:
legend_keys = ['Built-up', 'Bareland', 'Ag Feild2', 'Wetland', 'Ag Feild1']
legend_colors = ['#E99812','#4D167E' , '#9DC71A','#2BA0F1' ,'#66FA0D' ]

# Reclassify the map
result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map.addLayer(result, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')
Map

In [82]:
#Supervised Classification


#USDA NASS Cropland Data Layers for training and validation
USDA = ee.ImageCollection('USDA/NASS/CDL') \
                  .filter(ee.Filter.date('2017-01-01', '2018-01-01')) \
                  .first()

#cropLandcover
CDL = USDA.select('cultivated').clip(loc)
print(type(CDL))
bandNames = CDL.bandNames()
print('Band names: ', bandNames.getInfo())

<class 'ee.image.Image'>


ProtocolError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [76]:
Map = geemap.Map(center=(43.65, -95.17), zoom=10)
Map.addLayer(CDL, {}, 'CDL')
Map

Map(center=[43.65, -95.17], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

In [77]:
# Make the training dataset.
points = CDL.sample(**{
    'region': loc,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)

In [78]:
# Use these bands for prediction.
bands = ['R', 'G', 'B', "N"]


# This property of the table stores the land cover labels.
label = 'cultivated'

# Overlay the points on the imagery to get training.
training = median.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [79]:
# Classify the image with the same bands used for training.
result = median.select(bands).classify(trained)

# Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

Map(center=[43.65, -95.17], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

In [80]:
classifier = trained
train_accuracy = classifier.confusionMatrix()
train_accuracy.getInfo()

[[0, 0, 0], [0, 992, 17], [0, 1, 3990]]

In [69]:
train_accuracy.accuracy().getInfo()

0.9964

In [ ]:
train_accuracy.producersAccuracy().getInfo()

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

In [65]:
split = 0.7
sample = training.randomColumn()
validation = sample.filter(ee.Filter.gte('random', split))
validated = validation.classify(trained)
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '4_0',
 'properties': {'B': 112,
  'G': 153,
  'N': 124,
  'R': 138,
  'classification': 2,
  'cultivated': 2,
  'random': 0.7263330034630829}}

In [66]:
test_accuracy = validated.errorMatrix('cultivated', 'classification')
test_accuracy.getInfo()

[[0, 0, 0], [0, 290, 7], [0, 0, 1255]]

In [67]:
test_accuracy.accuracy().getInfo()

0.9954896907216495

In [68]:
test_accuracy.kappa().getInfo()

0.9852943565198088

In [ ]:
test_accuracy.producersAccuracy().getInfo()

In [ ]:
test_accuracy.consumersAccuracy().getInfo()

In [ ]:
# out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
# out_file = os.path.join(out_dir, 'cluster.tif')
# geemap.ee_export_image(result, filename=out_file, scale=90)

In [ ]:
# def stackCollection(collection):
#   # Create an initial image.
#   first = ee.Image(collection.first()).select([])

#   # Write a function that appends a band to an image.
#   def appendBands(image, previous):
#     return ee.Image(previous).addBands(image)
  
#   return ee.Image(collection.iterate(appendBands, first))


# geemap.ee_to_csv(ee_object, filename, selectors)

In [ ]:
# layers= stackCollection(collection)
# type(layers)

In [ ]:
# band_arrs = np.array(ndvi)

In [ ]:

# # Transfer the arrays from server to client and cast as np array.
# np_arr_b4 = np.array(band_arrs)

# print(np_arr_b4.shape)

In [ ]:

# mask = collection.eq(1)

# # Mask and mosaic visualization images.  The last layer is on top.
# mosaic = ee.ImageCollection([
#   # NDWI > 0.5 is water.  Visualize it with a blue palette.
#   ndwi.updateMask(ndwi.gte(0.5)).visualize(ndwiViz),
#   # NDVI > 0.2 is vegetation.  Visualize it with a green palette.
#   ndvi.updateMask(ndvi.gte(0.2)).visualize(ndviViz),
#   # Visualize bare areas with shadow (bare2 but not bare1) as gray.
#   bare2.updateMask(bare2.And(bare1.Not())).visualize(**{'palette': ['AAAAAA']}),
#   # Visualize the other bare areas as white.
#   bare1.updateMask(bare1).visualize(**{'palette': ['FFFFFF']}),
# ]).mosaic()
# Map.addLayer(mosaic, {}, 'Visualization mosaic')
# Map


In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads/GIS8990')

In [ ]:
#geemap.ee_export_image(naip, filename=naip, scale=10, region=MinnesotaBoundary, file_per_band=False)

In [ ]:
geemap.ee_export_image_collection(collection, out_dir=out_dir)